In [1]:
# import packages
import requests
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession


# Loading Libraries

In [205]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import csv
import json
import numpy as np
import seaborn as sns

# Initiate Driver to Scroll through page

In [351]:
driver = webdriver.Chrome('/Users/tracynguyen/Applications/chromedriver')
# url = 'https://shop.lululemon.com/c/women/_/N-7vf'
url = 'https://shop.lululemon.com/c/women-maintops/_/N-815'

driver.get(url)  # open WMTM page
i = 0
while True:
    try:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight + 1000);')  
        time.sleep(15)  
        button = driver.find_element_by_xpath("//span[text()='View more products']")
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()  # move button into view
        button.click()  
    
    except NoSuchElementException:  # all products loaded
        break
       
# driver.page_source  # return HTML text

## Get unique product urls
- ignore different colors

In [352]:
product_urls = [(i.get_attribute('href')).split("?", 1)[0] for i in driver.find_elements_by_xpath('.//div[@class="product-tile"]//a')]
product_urls = list(dict.fromkeys(product_urls))
product_urls

['https://shop.lululemon.com/p/tops-long-sleeve/Swiftly-Tech-LS-2/_/prod9750543',
 'https://shop.lululemon.com/p/women-tanks/Align-Tank-Diamond-Dye/_/prod10000326',
 'https://shop.lululemon.com/p/women-tanks/Align-Tank/_/prod9600539',
 'https://shop.lululemon.com/p/women-tanks/Blissful-Bend-Tank-Top/_/prod10190150',
 'https://shop.lululemon.com/p/women-tanks/Gather-and-Grow-Tank/_/prod10400588',
 'https://shop.lululemon.com/p/tops-long-sleeve/Gather-and-Grow-Long-Sleeve/_/prod10390404',
 'https://shop.lululemon.com/p/tops-long-sleeve/Ever-Ready-Long-Sleeve/_/prod9960686',
 'https://shop.lululemon.com/p/tops-long-sleeve/Back-In-Action-Ls/_/prod8555454',
 'https://shop.lululemon.com/p/tops-long-sleeve/LA-Back-in-Action-Long-Sleeve/_/prod9750526',
 'https://shop.lululemon.com/p/tops-long-sleeve/Rest-Less-Pullover/_/prod5020032',
 'https://shop.lululemon.com/p/tops-short-sleeve/Love-Crew-III/_/prod8690612',
 'https://shop.lululemon.com/p/women-tanks/Power-Pivot-Tank-Rib/_/prod9750455',
 'h

In [353]:
len(product_urls)

165

## Get product attributes

In [354]:
product_attributes = [ i.get_attribute('data-lulu-attributes') for i in driver.find_elements_by_xpath('.//h3[@class="product-tile__product-name lll-text-body-1"]//a')]
product_attributes

['{"type":"product","product":{"name":"Swiftly%20Tech%20Long%20Sleeve%202%2E0","skuID":"us_122876068","productID":"prod9750543","categoryUnifiedID":"swiftly-tech-ls-2","color":"45857","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":1},"version":"2.1"}',
 '{"type":"product","product":{"name":"lululemon%20Align%E2%84%A2%20Tank%20%2ADiamond%20Dye","skuID":"us_124975727","productID":"prod10000326","categoryUnifiedID":"align-tank-diamond-dye","color":"47168","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":2},"version":"2.1"}',
 '{"type":"product","product":{"name":"lululemon%20Align%E2%84%A2%20Tank","skuID":"us_127450385","productID":"prod9600539","categoryUnifiedID":"align-tank","color":"8313","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":4},"version":"2.1"}',
 '{"type":"product","product":{"name":"Blissful%20Bend%20Tank%20Top","skuID":"us_124

## Get product name

In [355]:
product_name = [ urllib.parse.unquote(json.loads(i.get_attribute('data-lulu-attributes'))['product']['name']) for i in driver.find_elements_by_xpath('.//h3[@class="product-tile__product-name lll-text-body-1"]//a')]
product_name

['Swiftly Tech Long Sleeve 2.0',
 'lululemon Align™ Tank *Diamond Dye',
 'lululemon Align™ Tank',
 'Blissful Bend Tank Top',
 'Gather and Grow Tank',
 'Gather and Grow Long Sleeve',
 'Ever Ready Long Sleeve',
 'Back In Action Long Sleeve',
 'LA Back in Action Long Sleeve',
 'Rest Less Pullover',
 'Love Crew III',
 'Power Pivot Tank *Rib',
 'Swiftly Tech Racerback 2.0',
 'Swiftly Tech Long Sleeve 2.0 *Race Length',
 'Swiftly Tech Long Sleeve 2.0 *MultiDye',
 'Rise and Run Short Sleeve',
 'Long Distance Short Sleeve',
 'All Yours Tee',
 'Time To Restore Short Sleeve',
 'Love Crew *Splatter',
 'Back in Action Long Sleeve *Rulu',
 'LA Split Shoulder Muscle Tank',
 'LA Mock Neck Layered Crop Tank',
 'Swiftly Tech Short Sleeve 2.0',
 'Love Tank *Pleated',
 'Ebb To Street Tank *Light Support For B/C Cup',
 'All Yours Tank',
 'Cool Racerback II *Nulu',
 'Love Tee V',
 'All Tied Up Tank',
 'Run Briskly 1/2 Zip',
 'Swiftly Breathe Long Sleeve',
 'Sculpt Tank',
 'Swiftly Breathe Long Sleeve Squad

## Get product price

In [356]:
price = [i.get_attribute("innerText").replace("\xa0\n", "").replace("\xa0-\xa0", "-") for i in driver.find_elements_by_xpath('.//span[@class="price-1SDQy price"]')]
price

['$78',
 '$64',
 '$58',
 '$68',
 '$54',
 '$80',
 '$58',
 '$58',
 '$58',
 '$88',
 '$48',
 '$58',
 '$58',
 '$78',
 '$88',
 '$68',
 '$58',
 '$58',
 '$58',
 '$54',
 '$88',
 '$58',
 '$68',
 '$68',
 '$38',
 '$58',
 '$44',
 '$48',
 '$48',
 '$44',
 '$98',
 '$78',
 '$58',
 '$78',
 '$48',
 '$68',
 '$48',
 '$68',
 '$58',
 '$52',
 '$68',
 '$68',
 '$48',
 '$58',
 '$58',
 '$88',
 '$58',
 '$78',
 '$44',
 '$52',
 '$78',
 '$58',
 '$68',
 '$58',
 '$78',
 '$48',
 '$68',
 '$68',
 '$78',
 '$78',
 '$58',
 '$48',
 '$88',
 '$88',
 '$52',
 '$48',
 '$48',
 '$88',
 '$65',
 '$68',
 '$58',
 '$58',
 '$58',
 '$58',
 '$48',
 '$58',
 '$58',
 '$58',
 '$38',
 '$68',
 '$68',
 '$58',
 '$58',
 '$48',
 '$58',
 '$68',
 '$78',
 '$78',
 '$78',
 'Sale Price$39-$59Regular Price$68',
 'Sale Price$64Regular Price$78',
 'Sale Price$39Regular Price$58',
 'Sale Price$29Regular Price$44',
 'Sale Price$39-$49Regular Price$58',
 'Sale Price$29-$34Regular Price$44',
 'Sale Price$59-$69Regular Price$78',
 'Sale Price$64Regular Price$78',


# Make dataframe and clean it up

In [358]:
df = pd.DataFrame()
df['product_name'] = product_name
df['price'] = price
df['url'] = product_urls

new = df["price"].str.split("Regular Price", n = 1, expand = True)
df["current_price"] = new[0].str.replace('Sale Price','')
df["old_price"]= new[1] 

df = df.drop(['price'], axis = 1)
df

,product_name,url,current_price,old_price
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,$78,None
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,$64,None
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,$58,None
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,$68,None
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,$54,None
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,$80,None
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,$58,None
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,$58,None
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,$58,None
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,$88,None


In [304]:
df.to_csv("20210120_product_price.csv")

pickle.dump(df, open( "20210123_product_price", "wb" ) )

## Read in dataframe with product urls

In [158]:
df = pd.read_csv("20210120_product_price.csv")
df = df.drop(['Unnamed: 0'], axis = 1)
## clean up price

df['current_price'] = df['current_price'].str.split('-').str[0].str.replace("$", "")
df

,product_name,url,current_price,old_price
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,NaN
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,NaN
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,NaN
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,NaN
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,NaN
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,80,NaN
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,NaN
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,NaN
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,NaN
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,88,NaN


In [159]:
df['old_price'] = df['old_price'].fillna(df['current_price'])
df['old_price'] = df['old_price'].str.split('-').str[0].str.replace("$", "")

df['current_price'] = pd.to_numeric(df['current_price'])
df['old_price'] = pd.to_numeric(df['old_price'])

df


,product_name,url,current_price,old_price
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,78
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,64
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,58
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,68
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,54
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,80,80
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,88,88


In [ ]:
product_urls = list(df['url'])
product_urls

# Review of each url

In [12]:
# driver.quit()
driver = webdriver.Chrome('/Users/tracynguyen/Applications/chromedriver')
product_index = 1

cols = ['url', 'stars', 'material']
lst = []

t = 0
for url in product_urls:
    stars = []
    material = []
    
    driver.get(url)
    
    i = 0
    while material == []:
        material = [i.get_attribute("innerHTML").replace(",", "").strip() for i in driver.find_elements_by_xpath('.//dd[@class="product-education-accordions__attributes__item__list"]')]
        i+=1
        if(i==4):
            material = ['None listed']

    i = 0
    while stars == []:
        if i == 10:
            stars = ['0', '0', '0', '0', '0']
            break
        try:
            button = driver.find_element_by_xpath("//span[text()='Reviews']")
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()  # move button into view
            button.click()  
            stars = [i.get_attribute("data-bv-histogram-rating-count") for i in driver.find_elements_by_xpath('.//div[@class="bv-inline-histogram-ratings-bar"]')]      
            i+=1
        except: 
            i+=1
            

    lst.append([url, stars, material])
    
    t+=1
    
    if t%5 == 0:
        print(t)
    

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165


In [305]:
df1 = pd.DataFrame(lst, columns=cols)
df1
df1.to_csv("20210120_product_review_material.csv")

pickle.dump(df1, open( "20210120_product_review_material", "wb" ) )

In [15]:
pd.options.display.max_rows = None
display(df1)

,url,stars,material
0,https://shop.lululemon.com/p/tops-long-sleeve/...,"[330, 109, 62, 75, 71]","[54% Nylon, 40% Recycled polyester, 3% Elastan..."
1,https://shop.lululemon.com/p/women-tanks/Align...,"[1058, 282, 161, 90, 65]","[89% Nylon, 11% Elastane, 80% Nylon, 20% Lycra..."
2,https://shop.lululemon.com/p/women-tanks/Align...,"[1058, 282, 161, 90, 65]",[None listed]
3,https://shop.lululemon.com/p/women-tanks/Bliss...,"[1, 1, 1, 1, 0]","[89% Nylon, 11% Elastane, 56% Polyester, 33% C..."
4,https://shop.lululemon.com/p/women-tanks/Gathe...,"[0, 1, 1, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]"
5,https://shop.lululemon.com/p/tops-long-sleeve/...,"[0, 0, 0, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]"
6,https://shop.lululemon.com/p/tops-long-sleeve/...,"[31, 4, 4, 3, 0]","[92% Pima cotton, 8% Lycra® elastane]"
7,https://shop.lululemon.com/p/tops-long-sleeve/...,"[462, 65, 56, 76, 72]","[92% Pima cotton, 8% Lycra® elastane]"
8,https://shop.lululemon.com/p/tops-long-sleeve/...,"[5, 1, 1, 1, 1]","[92% Pima cotton, 6% Lycra® elastane, 1% Polye..."
9,https://shop.lululemon.com/p/tops-long-sleeve/...,"[306, 78, 46, 57, 55]","[78% Nylon, 18% Polyester, 4% Elastane]"


In [283]:
merge_df = pd.merge(df, df1, on = 'url')
merge_df

,product_name,url,current_price,old_price,stars,material
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,78,"[330, 109, 62, 75, 71]","[54% Nylon, 40% Recycled polyester, 3% Elastan..."
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,64,"[1058, 282, 161, 90, 65]","[89% Nylon, 11% Elastane, 80% Nylon, 20% Lycra..."
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,58,"[1058, 282, 161, 90, 65]",[None listed]
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,68,"[1, 1, 1, 1, 0]","[89% Nylon, 11% Elastane, 56% Polyester, 33% C..."
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,54,"[0, 1, 1, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]"
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,80,80,"[0, 0, 0, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]"
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[31, 4, 4, 3, 0]","[92% Pima cotton, 8% Lycra® elastane]"
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[462, 65, 56, 76, 72]","[92% Pima cotton, 8% Lycra® elastane]"
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[5, 1, 1, 1, 1]","[92% Pima cotton, 6% Lycra® elastane, 1% Polye..."
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,88,88,"[306, 78, 46, 57, 55]","[78% Nylon, 18% Polyester, 4% Elastane]"


In [284]:
merge_df['type'] = merge_df['url'].str.split('/').str[4]
merge_df
# tmp_urls[1].split('/')[4]

,product_name,url,current_price,old_price,stars,material,type
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,78,"[330, 109, 62, 75, 71]","[54% Nylon, 40% Recycled polyester, 3% Elastan...",tops-long-sleeve
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,64,"[1058, 282, 161, 90, 65]","[89% Nylon, 11% Elastane, 80% Nylon, 20% Lycra...",women-tanks
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,58,"[1058, 282, 161, 90, 65]",[None listed],women-tanks
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,68,"[1, 1, 1, 1, 0]","[89% Nylon, 11% Elastane, 56% Polyester, 33% C...",women-tanks
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,54,"[0, 1, 1, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]",women-tanks
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,80,80,"[0, 0, 0, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[31, 4, 4, 3, 0]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[462, 65, 56, 76, 72]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[5, 1, 1, 1, 1]","[92% Pima cotton, 6% Lycra® elastane, 1% Polye...",tops-long-sleeve
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,88,88,"[306, 78, 46, 57, 55]","[78% Nylon, 18% Polyester, 4% Elastane]",tops-long-sleeve


In [213]:
merge_df.describe()

,current_price,old_price
count,165.000000,165.000000
mean,55.139394,66.157576
std,16.165643,18.213822
min,24.000000,38.000000
25%,44.000000,54.000000
50%,58.000000,58.000000
75%,68.000000,78.000000
max,98.000000,148.000000


In [285]:
def add_reviews(stars):
    total = 0
    for x in stars:
        total+=int(x)
    return(total)

def avg_stars(stars):
    i = 5
    score = 0
    total = 0
    for x in stars: 
        score +=(int(x)*i)
        total+=int(x)
        i-=1

    if score == 0 | total == 0:
        return(0)
    return(score/total)

def split_percent(material):
    if material == ['None listed']:
        return(material)
    
    new_list = []
    for x in material:
        new_list.append(x.split('% ')[1])
        
    new_list = sorted(list(set(new_list)))
    return(new_list)


        

merge_df['total_reviews']  = merge_df['stars'].apply(lambda x: add_reviews(x))
merge_df['avg_rating'] = merge_df['stars'].apply(lambda x: avg_stars(x))
merge_df['material_clean'] = merge_df['material'].apply(lambda x: split_percent(x))
merge_df['material_clean_list'] = [','.join(map(str, l)) for l in merge_df['material_clean']]
merge_df[['five_stars','four_stars', 'three_stars', 'two_stars', 'one_stars']] = pd.DataFrame(merge_df.stars.tolist(), index= df.index)

merge_df

# merge_df.to_csv("20210123_product_review_material.csv")

In [288]:
merge_df.head()

,product_name,url,current_price,old_price,stars,material,type,total_reviews,avg_rating,material_clean,material_clean_list,five_stars,four_stars,three_stars,two_stars,one_stars
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,78,"[330, 109, 62, 75, 71]","[54% Nylon, 40% Recycled polyester, 3% Elastan...",tops-long-sleeve,647,3.853168,"[Elastane, Nylon, Recycled polyester, X-static...","Elastane,Nylon,Recycled polyester,X-static® nylon",330,109,62,75,71
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,64,"[1058, 282, 161, 90, 65]","[89% Nylon, 11% Elastane, 80% Nylon, 20% Lycra...",women-tanks,1656,4.315217,"[Elastane, Lycra® elastane, Nylon]","Elastane,Lycra® elastane,Nylon",1058,282,161,90,65
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,58,"[1058, 282, 161, 90, 65]",[None listed],women-tanks,1656,4.315217,[None listed],None listed,1058,282,161,90,65
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,68,"[1, 1, 1, 1, 0]","[89% Nylon, 11% Elastane, 56% Polyester, 33% C...",women-tanks,4,3.500000,"[Coolmax® polyester, Elastane, Lycra® elastane...","Coolmax® polyester,Elastane,Lycra® elastane,Ny...",1,1,1,1,0
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,54,"[0, 1, 1, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]",women-tanks,2,3.500000,"[Lycra® elastane, Pima cotton]","Lycra® elastane,Pima cotton",0,1,1,0,0


In [294]:
material_list = ','.join(merge_df['material_clean_list'].unique())
material_list

'Elastane,Nylon,Recycled polyester,X-static® nylon,Elastane,Lycra® elastane,Nylon,None listed,Coolmax® polyester,Elastane,Lycra® elastane,Nylon,Polyester,Lycra® elastane,Pima cotton,Lycra® elastane,Pima cotton,Polyester,Viscose,Elastane,Nylon,Polyester,Lycra® elastane,Nylon,Elastane,Nylon,Polyester,X-static® nylon,Nylon,Xtra life lycra® elastane,Lycra® elastane,Recycled nylon,X-static® nylon,Lycra® elastane,Lyocell,Pima cotton,Lycra® elastane,Modal,Elastane,Nylon,Coolmax® polyester,Lycra® elastane,Nylon,Polyester,Elastane,Nylon,Xtra life lycra® elastane,Nylon,Polyester,X-static® nylon,Cotton,Nylon,Polyester,Elastane,Lycra® elastane,Nylon,Polyester,Lycra® elastane,Nylon,Xtra life lycra® elastane,Polyester,Viscose,Elastane,Lenzing® modal,Elastane,Nylon,Polyester,Xtra life lycra® elastane,Lycra® elastane,Polyester,Elastane,Lurex® metallic,Nylon,Polyester,Elastane,Modal,Silk,Elastane,Nylon,Wool,Lycra® elastane,Lyocell,Polyester,Cotton,Lyocell,Polyester,Elastane,Recycled nylon,Elastane,Line

In [303]:
set(list(material_list.split(",")))

{'Coolmax® polyester',
 'Cotton',
 'Elastane',
 'Lenzing® modal',
 'Linen',
 'Lurex® metallic',
 'Lycra® elastane',
 'Lyocell',
 'Modal',
 'None listed',
 'Nylon',
 'Pima cotton',
 'Polyester',
 'Recycled nylon',
 'Recycled polyester',
 'Silk',
 'Tencel® lyocell',
 'Viscose',
 'Wool',
 'X-static® nylon',
 'Xtra life lycra® elastane'}

In [306]:
import pickle
pickle.dump(merge_df, open( "20210123_product_review_material", "wb" ) )

In [307]:
df = pickle.load( open( "20210123_product_review_material", "rb" ) )
df

,product_name,url,current_price,old_price,stars,material,type,total_reviews,avg_rating,material_clean,material_clean_list,five_stars,four_stars,three_stars,two_stars,one_stars
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,78,78,"[330, 109, 62, 75, 71]","[54% Nylon, 40% Recycled polyester, 3% Elastan...",tops-long-sleeve,647,3.853168,"[Elastane, Nylon, Recycled polyester, X-static...","Elastane,Nylon,Recycled polyester,X-static® nylon",330,109,62,75,71
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,64,64,"[1058, 282, 161, 90, 65]","[89% Nylon, 11% Elastane, 80% Nylon, 20% Lycra...",women-tanks,1656,4.315217,"[Elastane, Lycra® elastane, Nylon]","Elastane,Lycra® elastane,Nylon",1058,282,161,90,65
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,58,58,"[1058, 282, 161, 90, 65]",[None listed],women-tanks,1656,4.315217,[None listed],None listed,1058,282,161,90,65
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,68,68,"[1, 1, 1, 1, 0]","[89% Nylon, 11% Elastane, 56% Polyester, 33% C...",women-tanks,4,3.500000,"[Coolmax® polyester, Elastane, Lycra® elastane...","Coolmax® polyester,Elastane,Lycra® elastane,Ny...",1,1,1,1,0
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,54,54,"[0, 1, 1, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]",women-tanks,2,3.500000,"[Lycra® elastane, Pima cotton]","Lycra® elastane,Pima cotton",0,1,1,0,0
5,Gather and Grow Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,80,80,"[0, 0, 0, 0, 0]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve,0,0.000000,"[Lycra® elastane, Pima cotton]","Lycra® elastane,Pima cotton",0,0,0,0,0
6,Ever Ready Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[31, 4, 4, 3, 0]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve,42,4.500000,"[Lycra® elastane, Pima cotton]","Lycra® elastane,Pima cotton",31,4,4,3,0
7,Back In Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[462, 65, 56, 76, 72]","[92% Pima cotton, 8% Lycra® elastane]",tops-long-sleeve,731,4.051984,"[Lycra® elastane, Pima cotton]","Lycra® elastane,Pima cotton",462,65,56,76,72
8,LA Back in Action Long Sleeve,https://shop.lululemon.com/p/tops-long-sleeve/...,58,58,"[5, 1, 1, 1, 1]","[92% Pima cotton, 6% Lycra® elastane, 1% Polye...",tops-long-sleeve,9,3.888889,"[Lycra® elastane, Pima cotton, Polyester, Visc...","Lycra® elastane,Pima cotton,Polyester,Viscose",5,1,1,1,1
9,Rest Less Pullover,https://shop.lululemon.com/p/tops-long-sleeve/...,88,88,"[306, 78, 46, 57, 55]","[78% Nylon, 18% Polyester, 4% Elastane]",tops-long-sleeve,542,3.964945,"[Elastane, Nylon, Polyester]","Elastane,Nylon,Polyester",306,78,46,57,55
